In [1]:
import pandas as pd, numpy as np, json, re, pickle
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [2]:
con_df = pd.read_json(r'C:\Users\Gabe\Documents\Bellevue University\Data Mining\Week 9 & 10\categorized-comments.jsonl', lines=True)
con_df.head()

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [3]:
def clean_text(text):
    """
    Remove punctuations and special characters, makes lower case
    Args: text 
    Output: text
    """
    
    text=text.lower()
    text=re.sub('</?.*?>',' <>', text)
    text=re.sub('\\d|\\W+|_',' ',text)
    text=re.sub('[^a-zA-Z]'," ", text)
    
    return text
# Create stop words list

stop_words = stopwords.words('english')

In [4]:
size = 30000    # sample size
replace = True  # with replacement
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]

comments = con_df.groupby('cat', as_index=False).apply(fn)

# free up memory

del con_df

comments['txt'] = comments['txt'].apply(lambda x:clean_text(x))
comments.reset_index(drop=True, inplace=True)

comments.head()

,cat,txt
0,science_and_technology,typical r android dismissing content just beca...
1,science_and_technology,you might need a new phone then lol
2,science_and_technology,i have no problems reaching that position on m...
3,science_and_technology,well i mean sure i can believe that on the ...
4,science_and_technology,gt weren t there reports of gmail adding snoo...


In [5]:
from keras.layers import Dense
from keras.models import Sequential

N_FEATURES = 5000
N_CLASSES = 4

def build_network():
    nn = Sequential()
    nn.add(Dense(500, activation='relu', input_shape=(N_FEATURES,)))
    nn.add(Dense(150, activation='relu'))
    nn.add(Dense(N_CLASSES, activation='softmax'))
    nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return nn


In [6]:
from sklearn.model_selection import train_test_split
sentences = comments['txt'].values
y = comments['cat'].values
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size = 0.25, random_state=1000)


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)
vectorizer.transform(sentences).toarray()

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

In [8]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

C:\Users\Gabe\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Gabe\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy: 0.7780888888888889


In [9]:
from keras.models import Sequential
from keras import layers

input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [10]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())
print(X_train)
print(y_train)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                392950    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 392,961
Trainable params: 392,961
Non-trainable params: 0
_________________________________________________________________
None
  (0, 993)	1
  (0, 3414)	1
  (0, 4711)	1
  (0, 12122)	1
  (0, 16451)	1
  (0, 28161)	1
  (0, 30040)	1
  (0, 30181)	1
  (0, 34407)	1
  (1, 6222)	1
  (1, 11624)	1
  (1, 15446)	1
  (1, 20683)	1
  (1, 23451)	1
  (1, 27083)	1
  (1, 28120)	1
  (1, 30382)	1
  (1, 31739)	1
  (1, 38422)	1
  (1, 38984)	1
  (2, 1501)	1
  (2, 6154)	2
  (2, 8598)	1
  (2, 14057)	1
  (2, 15863)	1
  :	:
  (67499, 22880)	1
  (67499, 24137)	1
  (67499, 24919)	1
  (67499, 25210)	1
  (67499, 25220)	1
  (67499, 25505)	1
  (67499, 25510)	1

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
#print(classification_report(y_train, y_test))

In [12]:
#print(confusion_matrix(y_train, y_test))